# Visualization

Being able to view medical images is a vital skill when training medical imaging AI models. There is a lot that can go wrong if you don't have a good way to visualize your images: the images could be misaligned, you could be training on blank data, and so on. Viualization is also important for checking whether your model is producing accurate predictions.

The ants library provides all of the functionality that you need to get a good visual understanding of your images. That functionality is what you will learn in this chapter. By the end of it, you will feel comfortable plotting 2D and 3D images both on their own and with both continuous or discrete overlay images. 

Importantly, the visualizations you learn to create here will be publication-quality, meaning you can use these functions to generate graphics that are included in reports, presentations, and publications.

## Plotting a single 2D image

### Adding overlays

## Plotting a single 3D image

### Adding overlays

## Making grids of multiple images

## Summary

In this section, you learned how to plot single 2D and 3D medical images on their own and with various types of overlay images. Plotting single images with discrete overlays, for example, is something you will do a ton if you train models for segmentation or classification.

We also showed how to plot multiple images together in a grid. This is useful when you want to compare multiple images at once, of course. 